In [11]:
from datetime import datetime, timedelta, timezone
import threading
import time 


In [12]:
class Scheduler:
    """
    A class to represent a schedule for a job with a start and end date.
    
    Attributes:
    -----------
    threading : boolean 
        Whether or not to use threading while running the code 
    
    Methods:
    --------
    add_job():
        Adds a job for running it 
    run_all():
        Runs all jobs with ot without threading
    """

    def __init__(self, threading=False):
        self.threads = threading
        self.jobs = []

        
        # print('IN INIT OF SCHEDULER')




############################# METHODS #############################
    
    ###### add_job ####### 
    def add_job(self, job):
        # Assign Scheduler's start date and time zone to the job if not provided
        # if job.startdate is None:
        #     job.startdate = self.stdate
        #     job.startdate = job.startdate.astimezone(self.time_zone)
        #     print('Adding Scheduler startdate to job:', job.startdate)
        # 
        # if job.time_zone is None:
        #     job.time_zone = self.time_zone
        #     print('Adding Scheduler timezone to job:', job.time_zone)

        # Adding job to the scheduler
        self.jobs.append(job)
        return self



    ####### run_all ####### 
    def run_all(self):
        # running jobs simultaneously when threads is true
        if self.threads:
            threads = []
            for job in self.jobs:
                # calling the Job run method for each thread
                thread = threading.Thread(target=job.run)
                threads.append(thread)
                
                # starting the execution of the thread
                thread.start()
            
            # ensure that run_all doesn't finish until all jobs have completed their execution.
            for thread in threads:
                thread.join()
        else:
            for job in self.jobs:
                job.run()
                


In [13]:
# Example Job Class
class Job:
    """
    A class to run jobs with specified time 

    Attributes:
    -----------
    startdate : str or datetime
        The start date and time of job. If not provided, takes the current date and time
    time_zone : str
        The timezone of the schedule. If not provided takes the current timezone of the computer

    Methods:
    --------
    do():
        finds the job for running 
    run_all():
        Runs all jobs with ot without threading

    Raises:
    -------
    ValueError:
        If Startdate is less than the current date.
    """
    def __init__(self, startdate =None, time_zone=None):
        self.startdate = startdate if startdate else datetime.now().strftime('%Y-%m-%d %H:%M')
        self.startdate = datetime.strptime(self.startdate, '%Y-%m-%d %H:%M')
        self.time_zone =time_zone # timezone hour offset
        self.enddate = None # until which time does user wants to run job 
        self.func = None  # function
        self.name = None # name of function
        self.unit = None # time unit
        self.interval = None # interval between running jobs
        self.next_run = None #next run time
        self.args = ()  # Positional arguments to pass to the job function.
        self.kwargs = {}  # Keyword arguments to pass to the job function.


        # setting time_zone 
        if time_zone is not None:
            self.time_zone = timezone(timedelta(hours=time_zone))
            self.startdate = self.startdate.astimezone(self.time_zone)
            
        else:
            # use the system's local timezone if no offset is provided
            self.time_zone = datetime.now().astimezone().tzinfo
            self.startdate = self.startdate.astimezone(self.time_zone)



        if self.startdate.strftime('%Y-%m-%d %H:%M') < datetime.now().strftime('%Y-%m-%d %H:%M'):
            raise ValueError("Startdate cannot be less than the current date")


        # print(f'stdate and timezone::: {self.startdate}, {self.time_zone}')




    ############################# METHODS #############################
    #### second ###
    @property
    def second(self):
        self.unit = 'second'
        return self


    #### do ###
    def do(self, func, name):
        """
        Parameters
        ----------
        func : function
            The function to be searched and run.
        name : str
            The name of the function provided by the user.
        """
        # print('INDO')
        self.func = func
        self.name = name
        return self

    #### every ####
    def every(self, interval):
        """
        Parameters
        ----------
        interval : int
            interval in which code will run (used to calculate next run time) 
        """
        self.interval = interval
        return self

    def calculate_next_run(self, current_time):
        # Calculate the next run time based on the job's schedule.
        if self.unit == 'second':
            self.next_run = current_time + timedelta(seconds=self.interval)
        else:
            raise ValueError("Unsupported unit. Please extend the `calculate_next_run` method to support other units.")
        return self.next_run

    #### until ###
    def until(self, enddate):
        """
        Parameters
        ----------
        enddate : str
            The date and time until which the user wants to run the code. The format should be 'YYYY-MM-DD HH:MM:SS'.

        Raises
        ------
        ValueError
            If the startdate occurs after enddate
        """
        # print('INUNTILL')
        self.enddate = datetime.strptime(enddate, '%Y-%m-%d %H:%M')
        self.enddate = self.enddate.astimezone(self.time_zone)
        # 
        if self.enddate < self.startdate:
            raise ValueError('Startdate must be earlier than Enddate')
        return self

    def run(self, *args, **kwargs):
            if self.func:
                self.func(*args, **kwargs)                      
            else:
                raise AttributeError('Function must be passed as attribute for code to run')


In [14]:
# Example functions
def report1(num1=1,num2 = 2):
    print(num1+num2)

def report2():
    print("Function report2 is running.")

def report3():
    print("Function report3 is running.")

# create Job instances and associate functions with them
job1 = Job(startdate='2024-08-21 10:00',time_zone=4).do(report1, 'report1').every(1).second.until('2024-08-22 10:00')
job2 = Job(startdate='2024-08-21 10:00',time_zone=5).do(report2, 'report2').every(1).second.until('2024-08-22 10:00')
job3 = Job(startdate='2024-08-21 10:00',time_zone=10).do(report3, 'report3').every(2).second.until('2024-08-22 10:00')


scheduler = Scheduler(threading=False)
scheduler.add_job(job1)
scheduler.add_job(job2)
scheduler.add_job(job3)
scheduler.run_all()

3
Function report2 is running.
Function report3 is running.


In [20]:
class ParentClass:
    def __init__(self, startdate):
        self.shared_attribute = startdate

class ChildClass(ParentClass):
    def __init__(self, shared_attribute, child_attribute = None):
        if child_attribute is None:
            self.child_attribute = super().__init__(shared_attribute)
        self.child_attribute = child_attribute

    def display_attributes(self):
        return f"Child Attribute: {self.child_attribute}"

# Example usage
parent = ParentClass("Shared Value")
child = ChildClass(shared_attribute ="Shared Value")

print(child.display_attributes())


Child Attribute: None


In [23]:
class ParentClass:
    def __init__(self, startdate):
        self.shared_attribute = startdate

class ChildClass(ParentClass):
    def __init__(self, child_attribute=None):
        # Initialize the parent class
        super().__init__("Shared Value")  # Automatically take the value from the parent

        # If child_attribute is None, use shared_attribute
        if child_attribute is None:
            self.child_attribute = self.shared_attribute
        else:
            self.child_attribute = child_attribute

    def display_attributes(self):
        return f"Child Attribute: {self.child_attribute}, Shared Attribute: {self.shared_attribute}"

# Example usage
parent = ParentClass("Shared Value")
child = ChildClass()

print(child.display_attributes())  # Output: Child Attribute: Shared Value, Shared Attribute: Shared Value


Child Attribute: Shared Value, Shared Attribute: Shared Value


In [ ]:
# Example Job Class
class Job:
    """
    A class to run jobs with specified time 

    Attributes:
    -----------
    startdate : str or datetime
        The start date and time of job. If not provided, takes the current date and time
    time_zone : str
        The timezone of the schedule. If not provided takes the current timezone of the computer

    Methods:
    --------
    do():
        finds the job for running 
    run_all():
        Runs all jobs with ot without threading

    Raises:
    -------
    ValueError:
        If Startdate is less than the current date.
    """
    def __init__(self, startdate =None, time_zone=None):
        self.startdate = startdate if startdate else datetime.now().strftime('%Y-%m-%d %H:%M')
        self.startdate = datetime.strptime(self.startdate, '%Y-%m-%d %H:%M')
        self.time_zone =time_zone # timezone hour offset
        self.enddate = None # until which time does user wants to run job 
        self.func = None  # function
        self.name = None # name of function
        self.unit = None # time unit
        self.interval = None # interval between running jobs
        self.next_run = None #next run time
        self.args = ()  # Positional arguments to pass to the job function.
        self.kwargs = {}  # Keyword arguments to pass to the job function.


        # setting time_zone 
        if time_zone is not None:
            self.time_zone = timezone(timedelta(hours=time_zone))
            self.startdate = self.startdate.astimezone(self.time_zone)

        else:
            # use the system's local timezone if no offset is provided
            self.time_zone = datetime.now().astimezone().tzinfo
            self.startdate = self.startdate.astimezone(self.time_zone)



        if self.startdate.strftime('%Y-%m-%d %H:%M') < datetime.now().strftime('%Y-%m-%d %H:%M'):
            raise ValueError("Startdate cannot be less than the current date")


        # print(f'stdate and timezone::: {self.startdate}, {self.time_zone}')




    ############################# METHODS #############################
    #### second ###
    @property
    def second(self):
        self.unit = 'second'
        return self


    #### do ###
    def do(self, func, name):
        """
        Parameters
        ----------
        func : function
            The function to be searched and run.
        name : str
            The name of the function provided by the user.
        """
        # print('INDO')
        self.func = func
        self.name = name
        return self

    #### every ####
    def every(self, interval):
        """
        Parameters
        ----------
        interval : int
            interval in which code will run (used to calculate next run time) 
        """
        self.interval = interval
        return self

    def calculate_next_run(self, current_time):
        # Calculate the next run time based on the job's schedule.
        if self.unit == 'second':
            self.next_run = current_time + timedelta(seconds=self.interval)
        else:
            raise ValueError("Unsupported unit. Please extend the `calculate_next_run` method to support other units.")
        return self.next_run

    #### until ###
    def until(self, enddate):
        """
        Parameters
        ----------
        enddate : str
            The date and time until which the user wants to run the code. The format should be 'YYYY-MM-DD HH:MM:SS'.

        Raises
        ------
        ValueError
            If the startdate occurs after enddate
        """
        # print('INUNTILL')
        self.enddate = datetime.strptime(enddate, '%Y-%m-%d %H:%M')
        self.enddate = self.enddate.astimezone(self.time_zone)
        # 
        if self.enddate < self.startdate:
            raise ValueError('Startdate must be earlier than Enddate')
        return self

    def run(self, *args, **kwargs):
        if self.func:
            self.func(*args, **kwargs)
        else:
            raise AttributeError('Function must be passed as attribute for code to run')